# NLP TP 2 - POS Tagging em Português
César de Paula Morais - 2021031521

Nesse trabalho, implementarei um modelo de NLP para POS-Tagging em português, a partir da base de treinamento Mac-morpho.

Como vimos em aula, há diversas maneiras de fazer o POS Tagging. A que mais me interessou, por curiosidade do resultado e da implementação em si, é o **Encoder-Only Transformer** - pode ser que a acurácia não seja tão alta nesse método, mas é uma boa oportunidade de explorar o potencial dos **context-aware embeddings**! <br />
Além disso, algo que me motivou a usar esse método foram as [aulas do canal StatQuest sobre o assunto](https://www.youtube.com/watch?v=GDN649X_acE&ab_channel=StatQuestwithJoshStarmer), em que é mencionado o poderio muitas vezes subestimado desse método (em favor dos Decoder-Only Transformers, famosos pelo GPT).

Na prática, usarei o [BERTimbau](https://huggingface.co/neuralmind/bert-base-portuguese-cased), um modelo BERT pré-treinado em português, para gerar os context-aware embeddings. Em seguida, com o Tensorflow, criei uma camada densa, de dimensão *tamanho do embedding BERTimbau -> número total de classes*. A entrada dessa camada será uma frase (formada pelos embeddings), e a saída será uma softmax com a predição da classe. <br />
Ao final, iremos verificar os resultados para as diferentes classes e apontar forças/fraquezas do método, tanto como possibilidade de melhoria.

## Funções Auxiliares

Primeiramente, definiremos os imports, e definiremos o tokenizer e modelo do BERTimbau

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.metrics import classification_report
import os
import pickle

NUMBER_OF_LABELS = 30

tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased", max_length=512, truncation=True)
bert_model = AutoModel.from_pretrained("neuralmind/bert-base-portuguese-cased")

As funções abaixo são responsáveis por ler as labels do macmorpho (esse arquivo foi escrito à mão, com ajuda da documentação) e carregar esses dados em vetores ordenados de frases e suas labels, já em formato de número (a entrada da camada densa).

Um detalhe de implementação é que, na função `load_macmorpho`, já tokenizamos as palavras com o BERTimbau. Isso ocorre pois não necessariamente o modelo irá tokenizar as palavras a partir dos caracteres de espaço - pode ser que uma palavra seja tokenizada por duas ou mais. Caso isso ocorra, estamos marcando a segunda parte dessa palavra como "inutilizada". Tal tratamento ocorre para evitar problemas de dimensionalidade durante o treinamento.

In [2]:
def get_labels(labels_path="input/macmorpho-labels"):
    lines = []

    with open(labels_path, "r", encoding="utf-8") as file:
        for line in file:
            lines.append(line.strip())

    return lines

def load_macmorpho(file_path, tokenizer, label_list):
    sentences = []
    labels = []

    with open(file_path, "r", encoding="utf-8") as f:
        current_sentence = []
        current_label = []

        for line in f:
            line = line.strip()
            tokens = line.split(" ")

            for token in tokens:
                word, label = token.rsplit("_", 1)
                subwords = tokenizer.tokenize(word)  # Subword tokenization
                subword_count = len(subwords)

                current_sentence.extend(subwords)
                current_label.extend([label_list.index(label)] + [-1] * (subword_count - 1))  # -1 marks subwords that should not contribute to loss

            sentences.append(current_sentence)
            labels.append(current_label)
            current_sentence = []
            current_label = []

    return sentences, labels

A partir das frases e labels, então, geramos os embeddings, e depois faremos um tratamento para os tokens inválidos do passo anterior

In [3]:
def get_sentence_embeddings(sentences, labels):
    sentence_embeddings = []
    expanded_labels = []

    for sentence, label in zip(sentences, labels):
        input_ids = tokenizer(sentence, return_tensors='pt', is_split_into_words=True, padding=True, truncation=True)

        with torch.no_grad():
            outs = bert_model(**input_ids)
            token_embeddings = outs.last_hidden_state.squeeze(0)[1:-1]  # Remove [CLS] and [SEP]

        # Align labels with tokens
        word_ids = input_ids.word_ids() 
        expanded_label = []
        for word_id in word_ids:
            if word_id is None:
                continue
            elif expanded_label and word_id == expanded_label[-1]:
                expanded_label.append(-1)
            else:
                expanded_label.append(label[word_id])

        # Ensure embeddings and labels match
        if len(token_embeddings) != len(expanded_label):
            raise ValueError(f"Mismatch between tokens and labels: {len(token_embeddings)} vs {len(expanded_label)}")

        sentence_embeddings.append(token_embeddings)
        expanded_labels.extend(expanded_label)

    return torch.cat(sentence_embeddings).numpy(), np.array(expanded_labels)

## Treinamento
Vamos começar gerando os embeddings de treinamento. Essa é a fase com maior custo computacional, definitivamente. Por isso, optei por salvar os embeddings em um arquivo pickle, caso queira treinar novamente.

In [ ]:
print("Reading training file...")
training_sentence_tokens, training_labels = load_macmorpho("input/macmorpho-train.txt", tokenizer, get_labels())

if os.path.exists("output/training_embeddings.pkl"):
    print(f"Loading embeddings...")
    with open("output/training_embeddings.pkl", "rb") as f:
        data = pickle.load(f)
    X_train, y_train = data["embeddings"], data["labels"]
else:
    print(f"Computing embeddings...")
    X_train, y_train = get_sentence_embeddings(training_sentence_tokens, training_labels)

    with open("output/training_embeddings.pkl", "wb") as f:
        pickle.dump({"embeddings": X_train, "labels": y_train}, f)

Agora, vamos definir o modelo da camada densa. Novamente, tem as dimensões de entrada do tamanho do embedding, e sua ativação é uma softmax.

Os parâmetros (otimizador, learning_rate, epochs, etc) foram decididos de forma empírica. Foi usada uma split de validação de 20%.

In [5]:
model = Sequential([
    Dense(NUMBER_OF_LABELS, input_dim=768, activation='softmax')  # 768 is the size of the BERTimbau embedding
])

model.compile(optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Removes invalid tokes
valid_indices = y_train != -1
X_train = X_train[valid_indices]
y_train = y_train[valid_indices]

print("Starting training")
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

/home/cesar/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
W0000 00:00:1737140308.420241    1464 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Starting training
Epoch 1/10
17734/17734 ━━━━━━━━━━━━━━━━━━━━ 13s 724us/step - accuracy: 0.9283 - loss: 0.2808 - val_accuracy: 0.9442 - val_loss: 0.1981
Epoch 2/10
17734/17734 ━━━━━━━━━━━━━━━━━━━━ 12s 686us/step - accuracy: 0.9627 - loss: 0.1303 - val_accuracy: 0.9444 - val_loss: 0.1995
Epoch 3/10
17734/17734 ━━━━━━━━━━━━━━━━━━━━ 12s 688us/step - accuracy: 0.9642 - loss: 0.1241 - val_accuracy: 0.9465 - val_loss: 0.1959
Epoch 4/10
17734/17734 ━━━━━━━━━━━━━━━━━━━━ 12s 688us/step - accuracy: 0.9645 - loss: 0.1214 - val_accuracy: 0.9441 - val_loss: 0.2010
Epoch 5/10
17734/17734 ━━━━━━━━━━━━━━━━━━━━ 12s 685us/step - accuracy: 0.9653 - loss: 0.1198 - val_accuracy: 0.9465 - val_loss: 0.1969
Epoch 6/10
17734/17734 ━━━━━━━━━━━━━━━━━━━━ 12s 686us/step - accuracy: 0.9654 - loss: 0.1182 - val_accuracy: 0.9455 - val_loss: 0.2011
Epoch 7/10
17734/17734 ━━━━━━━━━━━━━━━━━━━━ 12s 689us/step - accuracy: 0.9658 - loss: 0.1163 - val_accuracy: 0.9463 - val_loss: 0.1995
Epoch 8/10
17734/17734 ━━━━━━━━━━━━━━

## Teste
O mesmo procedimento será feito no arquivo de teste.

In [6]:
print("Reading test file...")
testing_sentence_tokens, testing_labels = load_macmorpho("input/macmorpho-test.txt", tokenizer, get_labels())

if os.path.exists("output/testing_embeddings.pkl"):
    print(f"Loading embeddings...")
    with open("output/testing_embeddings.pkl", "rb") as f:
        data = pickle.load(f)
    X_test, y_test = data["embeddings"], data["labels"]
else:
    print(f"Computing embeddings...")
    X_test, y_test = get_sentence_embeddings(testing_sentence_tokens, testing_labels)

    with open("output/testing_embeddings.pkl", "wb") as f:
        pickle.dump({"embeddings": X_test, "labels": y_test}, f)

print("Testing...")
prediction = model.predict(X_test)
predicted_classes = np.argmax(prediction, axis=1)

Reading test file...
Loading embeddings...
Testing...
9264/9264 ━━━━━━━━━━━━━━━━━━━━ 5s 518us/step


Finalmente, obteremos o resultado com ajuda da função classification report do sklearn - e imprimindo com os nomes das classes do MacMorpho.

In [22]:
classes = get_labels()
mapped_y_test = [classes[i-1] for i in y_test]
mapped_predicted_classes = [classes[i-1] for i in predicted_classes]

print(classification_report(mapped_y_test, mapped_predicted_classes, zero_division=0))

              precision    recall  f1-score   support

         ADJ       0.66      0.86      0.75      5256
         ADV       0.84      0.68      0.75       227
  ADV-KS-REL       0.93      0.99      0.96     12328
         ART       0.87      0.97      0.92      4430
         CUR       0.88      1.00      0.94     26328
          IN       0.29      0.95      0.45     35414
          KC       0.85      0.91      0.88      2487
          KS       0.32      0.51      0.40        98
           N       0.57      0.94      0.71     15535
       NPROP       0.66      0.93      0.77      2481
         NUM       0.72      0.81      0.76      3541
         PCP       0.81      0.85      0.83      1081
        PDEN       0.90      0.97      0.93     16369
        PREP       0.64      0.94      0.76      3328
    PREP+ADV       0.31      0.87      0.46      8335
    PREP+ART       0.77      0.97      0.86      2792
 PREP+PRO-KS       1.00      0.00      0.00    122753
 PREP+PROADJ       0.44    

## Resultados
Em geral, tivemos uma acurácia de 56%, um valor que pode ser melhorado para POS Tagging. Mesmo assim, creio que o experimento foi bem-sucedido no aprendizado dos **Encoder-Only Transformers** - a implementação foi desafiadora e o uso do BERTimbau, modelo conhecido no ambiente de NLP, foi importante.

Percebe-se um desbalanço de classes - algumas possuem muitas instâncias (como PREP+PRO-KS), enquanto outras ocorrem raramente (PREP+PROPESS). É uma tendência do modelo acertar mais as classes mais dominantes. Ou seja, esse desbalanço pode ter feito o modelo tender a predizer mais uma classe do que outra. 

A comparação da macro average e weighted average da precisão também indica isso, já que a weighted average (que leva em consideração o número de instâncias) é maior.

## Análise do Método e Modelo
- **Pontos positivos:** o aprendizado desse tipo de transformer foi o principal ponto positivo! Além disso, os resultados foram interessantes, com acurácia beirando os 60%.
- **Pontos a melhorar**: a performance em classes menos significativas foi menor, no geral. Há técnicas para mitigação, como, por exemplo, o uso de pesos para classes durante o treinamento, ou fazer *undersampling/oversampling*, também no treinamento.
- **Passos extras**: outra forma de, talvez, melhorar a acurácia do modelo é alterar os parâmetros da camada densa e do treinamento, por meio de métodos como GridSearch.